<a href="https://colab.research.google.com/github/robocarstore/donkey-car-training-on-google-colab/blob/master/Donkey_Car_Training_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU and Tensorflow version
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [ ]:
!nvidia-smi

Verified tensorflow version: > 2.9 and <= 2.15

In [ ]:
import tensorflow
print(tensorflow.__version__)

## Git Clone the donkey repository

In [ ]:
!git clone https://github.com/robocarstore/donkeycar.git
%cd /content/donkeycar

In [ ]:
!git checkout v5

## Install donkey car

In [ ]:
# !pip3 install --upgrade pip setuptools wheel
!pip3 install -e .[pc]

## Create Project

In [ ]:
!donkey createcar --path /content/mycar

### Download your dataset

You can download your dataset to this Colab instance by the following command. You might obtain the uuid of the dataset from the meta.json under each tub.

In [ ]:
%cd /content/mycar/data
!curl  -O -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/63783f2f-9497-460b-a6ad-3607c20dcbff/data.tar.gz"


Use the following command to extract the dataset into a folder named tub_xxxxxxx

In [ ]:
!tar -xzf data.tar.gz

In [ ]:
!curl  -O -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/19007878-6909-4c46-b6da-59e95355b15a/data.tar.gz"
!tar -xzf data.tar.gz
!rm data.tar.gz

In [ ]:
!ls

Assuming the latest created folder is the tub_name.

In [ ]:
tub_name=!ls -td -- */ | head -n 1
tub_name = tub_name[0].strip('/')
tub_name

Check whether the data is there

In [ ]:
!ls {tub_name}

Check number of images in this tub

In [30]:
%cd /content/mycar/data/{tub_name}
!find . -type f -name "*.jpg" | wc -l

/content/mycar/data/tub_49_24-01-13
6623


### Generate a movie for preview

In [32]:
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis



  Cloning https://github.com/sctse999/keras-vis to /tmp/pip-req-build-8grlag6v
  Running command git clone --filter=blob:none --quiet https://github.com/sctse999/keras-vis /tmp/pip-req-build-8grlag6v
  Resolved https://github.com/sctse999/keras-vis to commit 93e1e803dc0779e25f95b6c1e871d499414e5404
  Preparing metadata (setup.py) ... done
  Created wheel for keras-vis: filename=keras_vis-0.5.0-py2.py3-none-any.whl size=30956 sha256=9e9222a99b8a759813da80613ef716d63a9e441f96c06a3d415e543b2db3aecb
  Stored in directory: /tmp/pip-ephem-wheel-cache-mzbrhzn7/wheels/76/f4/c7/79f07c4e592d76b0adc5dc32b610988e36a56e8d288d286612
Successfully built keras-vis


In [33]:
%cd /content/mycar
!donkey makemovie --tub ./data/{tub_name} --out {tub_name}.mp4 --start 1 --end 2000

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer retu

Download this movie

In [40]:
from google.colab import files

files.download(f"{tub_name}.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train your model

In [42]:
%cd /content/mycar

model_name = "model1"

!donkey train --tub ./data/{tub_name} --model models/{model_name}

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...
INFO:donkeycar.config:loading config file: ./config.py
INFO:donkeycar.config:loading personal config over-rides from ./myconfig.py
2024-01-16 07:33:33.014099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 07:33:33.014151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already b

### Quiz
1. How many records are used for training?
2. How many records are used for validation?
3. Does the sum of this number adds up to the number of images in the dataset?
4. What is the final lost value?
5. How long does this training last?
6. How many epoches in this training?
7. How many steps in this epoch?
8. What's the value of `MAX_EPOCHS` in `myconfig.py`?
9. Why the training stop before it reaches the 100th epoch?

Check if the model is generated



In [ ]:
!ls -alh /content/mycar/models/{model_name}

In [ ]:
%cd /content/mycar/models/{model_name}

import glob
file = glob.glob("*.png")

from IPython.display import Image
Image(file[0])

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [ ]:
from google.colab import files

files.download('./mypilot.h5')

Alternatively, you can copy the model back to Google Drive too

In [ ]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models/mypilot.h5

### Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

```
sftp pi@raspberry.local
cd mycar/models
put mypilot.h5
```

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Changing training parameters

### Change number of epoch

1. Edit `myconfig.py`
2. Change `MAX_EPOCHS` to 5, 10 , 20
3. Training the model again by executing the training script above
4. What is `MAX_EPOCHS` used for?
5. Why do we want to change `MAX_EPOCHS`?

### Change TRAIN_FILTER

Change TRAIN_FILTER in `myconfig.py` to > 0.1 and check the number of images used for training and validation

### Change Batch Size

1. Edit `myconfig.py`
2. Change `BATCH_SIZE` to 256
3. Training the model again by executing the training script above
4. Does the training go faster? Why?


## Bonus - Salient Object Visualization

In [ ]:
!pip install imageio-ffmpeg


In [ ]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/mypilot.h5 --type linear --salient

In [ ]:
from google.colab import files

%cd /content/mycar
!ls -ahl
files.download('tub_movie.mp4')